# SKEMA-TA1-UAZ-MORAE Demo (2022-12-08)

**Authors**: Justin Lieffers, Liang Zhang, Deepsana Shahi, Adarsh Pyarelal, Clayton Morrison

In [2]:
# Set the base URL
BASE_URL="http://localhost:8080"

In [3]:
from IPython.display import IFrame
IFrame(f"{BASE_URL}/docs/", width=700, height=350)

## Model role identification

One of the prerequisites for model role identification is the ability to efficiently query GroMEts, including path traversal queries. For this reason, we have adopted a graph database backend for storing the structure inherent in GroMEt JSON files, along with additional metadata to speed up queries.

Below, we demonstrate how to interact with this database using our REST API.

First, we perform our imports and set the base URL

In [5]:
import json
from requests import get, put, post, delete

Then, we check if the service is running.

In [6]:
get(f"{BASE_URL}/ping").text

'The SKEMA Rust web services are running.'

We then insert the model represented by the GroMEt JSON file `sir_core--Gromet-FN-auto.json` into the database. The operation returns the ID of the model that was just inserted - we will use this ID to query the model later.

In [23]:
with open('data/sir_core--Gromet-FN-auto.json') as f:
    r = post(f"{BASE_URL}/models", json=json.load(f))
    MODEL_ID = r.json()
    print(f"Model ID: {MODEL_ID}")

Model ID: 2708


We then get the IDs of all the models in the database to double check if our model ID is among them.

In [24]:
get(f"{BASE_URL}/models").json()

[2233, 2328, 2423, 2518, 2708]

We can run a query to get all the names of named `opi`s in the model.

In [25]:
get(f"{BASE_URL}/models/{MODEL_ID}/named_opis").json()

['s', 'i', 'r', 'beta', 'gamma', 'n']

We can also run a query to get all the names of named `opo`s in the model.

In [26]:
get(f"{BASE_URL}/models/{MODEL_ID}/named_opos").json()

['s_n', 'i_n', 'r_n', 'scale', 's', 'i', 'r']

We now also have preliminary support for converting the code graphs into AMR's. Where the example below is for a PetriNet version, as we ingested an SIR model. For RegNet's change PN -> RN

In [27]:
get(f"{BASE_URL}/models/{MODEL_ID}/PN").json()

{'name': 'mathml model',
 'schema': 'https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/petrinet_v0.1/petrinet/petrinet_schema.json',
 'schema_name': 'PetriNet',
 'description': 'This is a model from mathml equations',
 'model_version': '0.1',
 'model': {'states': [{'id': 'i', 'name': 'i'},
   {'id': 'r', 'name': 'r'},
   {'id': 's', 'name': 's'}],
  'transitions': [{'id': 'beta',
    'input': ['i', 's'],
    'output': ['i', 'i'],
    'grounding': None},
   {'id': 'gamma', 'input': ['i'], 'output': ['r'], 'grounding': None}]}}

Finally, we demonstrate the API endpoint for deleting a model.

In [28]:
delete(f"{BASE_URL}/models/{MODEL_ID}").text

'Model deleted'

Get the IDs of all the models in the database again to check that the model has been deleted.

In [29]:
get(f"{BASE_URL}/models").json()

[2233, 2328, 2423, 2518]

There is also support for converting mathml into AMR's, either PetriNets or RegNets again. Below is for regnets using lotka voltera mathml

In [63]:
mathml_vec = []

with open("data/lotka_voltera/mml_list.txt") as file:
    for line in file:
        mathml_vec.append(line.rstrip())

print(json.dumps(mathml_vec))

headers = {
    "Content-Type": "application/json"
}

r = put(f"{BASE_URL}/mathml/regnet", data=json.dumps(mathml_vec), headers=headers)
r.json()

["<math><mrow><mfrac><mrow><mi>d</mi><mi>x</mi></mrow><mrow><mi>d</mi><mi>t</mi></mrow></mfrac><mo>=</mo><mi>alpha</mi><mi>x</mi><mo>-</mo><mi>beta</mi><mi>x</mi><mi>y</mi></mrow></math>", "<math><mrow><mfrac><mrow><mi>d</mi><mi>y</mi></mrow><mrow><mi>d</mi><mi>t</mi></mrow></mfrac><mo>=</mo><mi>delta</mi><mi>x</mi><mi>y</mi><mo>-</mo><mi>gamma</mi><mi>y</mi></mrow></math>"]


{'name': 'Regnet mathml model',
 'schema': 'https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/regnet_v0.1/regnet/regnet_schema.json',
 'schema_name': 'regnet',
 'description': 'This is a Regnet model from mathml equations',
 'model_version': '0.1',
 'model': {'vertices': [{'id': 'x',
    'name': 'x',
    'rate_constant': 'alpha',
    'sign': True},
   {'id': 'y', 'name': 'y', 'rate_constant': 'gamma', 'sign': False}],
  'edges': [{'id': 'beta',
    'source': ['y'],
    'target': ['x'],
    'sign': False,
    'grounding': None,
    'properties': {'rate_constant': 'beta'}},
   {'id': 'delta',
    'source': ['x'],
    'target': ['y'],
    'sign': True,
    'grounding': None,
    'properties': {'rate_constant': 'delta'}}]}}

We can also generate for a petrinet, below is an example with sir mathml

In [64]:
mathml_vec = []

with open("data/simple_sir_v1/mml_list.txt") as file:
    for line in file:
        mathml_vec.append(line.rstrip())

print(json.dumps(mathml_vec))

headers = {
    "Content-Type": "application/json"
}

r = put(f"{BASE_URL}/mathml/petrinet", data=json.dumps(mathml_vec), headers=headers)
r.json()

["<math display=\"block\" style=\"display:inline-block;\"><mrow><mfrac><mrow><mi>d</mi><mi>S</mi></mrow><mrow><mi>d</mi><mi>t</mi></mrow></mfrac><mo>=</mo><mo>\u2212</mo><mi>\u03b2</mi><mi>S</mi><mi>I</mi></mrow></math>", "<math display=\"block\" style=\"display:inline-block;\"><mrow><mfrac><mrow><mi>d</mi><mi>I</mi></mrow><mrow><mi>d</mi><mi>t</mi></mrow></mfrac><mo>=</mo><mi>\u03b2</mi><mi>S</mi><mi>I</mi><mo>\u2212</mo><mi>\u03b3</mi><mi>I</mi></mrow></math>", "<math display=\"block\" style=\"display:inline-block;\"><mrow><mfrac><mrow><mi>d</mi><mi>R</mi></mrow><mrow><mi>d</mi><mi>t</mi></mrow></mfrac><mo>=</mo><mi>\u03b3</mi><mi>I</mi></mrow></math>"]


{'name': 'mathml model',
 'schema': 'https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/petrinet_v0.1/petrinet/petrinet_schema.json',
 'schema_name': 'PetriNet',
 'description': 'This is a model from mathml equations',
 'model_version': '0.1',
 'model': {'states': [{'id': 'I', 'name': 'I'},
   {'id': 'R', 'name': 'R'},
   {'id': 'S', 'name': 'S'}],
  'transitions': [{'id': 'β',
    'input': ['I', 'S'],
    'output': ['I', 'I'],
    'grounding': None},
   {'id': 'γ', 'input': ['I'], 'output': ['R'], 'grounding': None}]}}

Lastly we also have support to route the mathml based on the model type if given as json object as the following:

In [65]:
class MathmlModel:
    def __init__(self, mathml, model):
        self.mathml = mathml
        self.model = model
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)

mathml_vec = []

with open("data/lotka_voltera/mml_list.txt") as file:
    for line in file:
        mathml_vec.append(line.rstrip())

lotka_voltera = MathmlModel(mathml_vec, "regnet")

headers = {
    "Content-Type": "application/json"
}

print(lotka_voltera.toJSON())

r = put(f"{BASE_URL}/mathml/amr", data=lotka_voltera.toJSON(), headers=headers)
r.json()

{
    "mathml": [
        "<math><mrow><mfrac><mrow><mi>d</mi><mi>x</mi></mrow><mrow><mi>d</mi><mi>t</mi></mrow></mfrac><mo>=</mo><mi>alpha</mi><mi>x</mi><mo>-</mo><mi>beta</mi><mi>x</mi><mi>y</mi></mrow></math>",
        "<math><mrow><mfrac><mrow><mi>d</mi><mi>y</mi></mrow><mrow><mi>d</mi><mi>t</mi></mrow></mfrac><mo>=</mo><mi>delta</mi><mi>x</mi><mi>y</mi><mo>-</mo><mi>gamma</mi><mi>y</mi></mrow></math>"
    ],
    "model": "regnet"
}


{'name': 'Regnet mathml model',
 'schema': 'https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/regnet_v0.1/regnet/regnet_schema.json',
 'schema_name': 'regnet',
 'description': 'This is a Regnet model from mathml equations',
 'model_version': '0.1',
 'model': {'vertices': [{'id': 'y',
    'name': 'y',
    'rate_constant': 'gamma',
    'sign': False},
   {'id': 'x', 'name': 'x', 'rate_constant': 'alpha', 'sign': True}],
  'edges': [{'id': 'delta',
    'source': ['x'],
    'target': ['y'],
    'sign': True,
    'grounding': None,
    'properties': {'rate_constant': 'delta'}},
   {'id': 'beta',
    'source': ['y'],
    'target': ['x'],
    'sign': False,
    'grounding': None,
    'properties': {'rate_constant': 'beta'}}]}}